In [ ]:
#PyGSTi tools
import pygsti
#pre-built gateset to use
from pygsti.modelpacks import smq1Q_XYZI
from pygsti.data.dataset import DataSet
from pygsti.io.writers import write_dataset

#Rigetti tools
from pyquil import get_qc, Program
from pyquil.api import QCSClientConfiguration, local_forest_runtime
from pyquil.gates import RESET

configuration = QCSClientConfiguration.load()

#python helper libraries
from multiprocessing.pool import ThreadPool
import time
import numpy as np
import pickle

In [ ]:
target_model = smq1Q_XYZI.target_model()      # a Model object
prep_fiducials = smq1Q_XYZI.prep_fiducials()  # a list of Circuit objects
meas_fiducials = smq1Q_XYZI.meas_fiducials()  # a list of Circuit objects
germs = smq1Q_XYZI.germs()                    # a list of Circuit objects
maxLengths = [1,2,4,8,16,32]
exp_design = pygsti.protocols.StandardGSTDesign(target_model, prep_fiducials, meas_fiducials,
                                                germs, maxLengths)

exp_design.all_circuits_needing_data
circuits = list(exp_design.all_circuits_needing_data)

In [ ]:
pygsti.io.write_empty_protocol_data('experiment_data/rigetti_XYZI_data', exp_design, clobber_ok=True)
#Actual (`empty) dataset located at tutorial_files/test_gst_dir/data/dataset.txt

In [ ]:
qpu = get_qc("Aspen-11", client_configuration=configuration, execution_timeout = 100000, compiler_timeout = 100000)
#qpu = get_qc("9q-square-qvm", noisy=True, execution_timeout = 100000, compiler_timeout = 100000)

def circ_fname(circ):
    return "circuit_binaries/%s .circ"%str(circ).replace("-","").replace('\n','').replace('|','').replace(' ','').replace('G','')[8:]

shots = 1000
num_circs = len(circuits)

In [ ]:


start_time = time.time()
for (i,circ) in enumerate(circuits):
    prog = Program(circ.convert_to_quil()).wrap_in_numshots_loop(shots)
    executable = qpu.compile(prog)
    with open(circ_fname(circ), "wb") as f:
        pickle.dump(executable, f)
        
    print("finished ",i, " Done in ",(time.time()-start_time)/(i+1)*(num_circs-i), end='\r')
    
print(time.time() - start_time)

In [ ]:
results = []

def run(circ):
    with open(circ_fname(circ), "rb") as f:
        executable = pickle.load(f)
    print("ran")
    result = qpu.run(executable).readout_data.get("ro")
    zeros = len([i for i in result if i== [0]])
    return zeros

for (i,circ) in enumerate(circuits):
    print("Running ", i)
    results.append(run(circ))

In [ ]:
time_data = [0]*len(results)
data = DataSet()

for (circ, result) in zip(circuits, results):
    data.add_count_dict(circ, {'0':result, '1':shots-result})
    
write_dataset("experiment_data/rigetti_XYZI_data/data/dataset.txt", data)

In [ ]:
%%bash

tar -cvf second_experiment_run_data.tar experiment_data circuit_binaries